# followed this tutorial on towards data science website made by Jaemin Lee:
# https://towardsdatascience.com/twitter-sentiment-analysis-nlp-text-analytics-b7b296d71fce

# importing libraries and installing libraries

In [45]:
#most likely you don't have this library installed
!pip install tweet-preprocessor
#re : regular expression
import re
# used to remove special characters
import sklearn
import numpy as np
import pandas as pd
from sklearn import svm
import preprocessor as p
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# importing data

In [46]:
train=pd.read_csv(r'C:\Users\user\dataset\train.csv')
test=pd.read_csv(r'C:\Users\user\dataset\test.csv')

# Exploring the data

In [47]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [48]:
test.tail()

,id,tweet
17192,49155,thought factory: left-right polarisation! #tru...
17193,49156,feeling like a mermaid ð #hairflip #neverre...
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,"happy, at work conference: right mindset leads..."
17196,49159,"my song ""so glad"" free download! #shoegaze ..."


# checking for missing values

In [49]:
train.isnull().values.any()

False

# defining symbols that we want to replace

In [50]:
REPLACE_NO_SPACE=re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE=re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

# function to clean the dataset using tweet-preprocessor and re

In [51]:
def clean_tweets(ds):
  Arr=[]
  for l in ds:
    # tweet-preprocessor
    tmp = p.clean(l)
    # re
    tmp=REPLACE_NO_SPACE.sub("",tmp.lower()) # convert all tweets to lower cases
    tmp=REPLACE_WITH_SPACE.sub(" ",tmp)
    Arr.append(tmp)
  return Arr

# cleaning training data

In [52]:
train["tweet"]=pd.DataFrame(clean_tweets(train["tweet"]))
test["tweet"]=pd.DataFrame(clean_tweets(test["tweet"]))

# spliting Testing and Training datasets

In [53]:
# per default 75%training and 25%test
x_train,x_test,y_train,y_test=train_test_split(train.tweet.values,
                                               train.label.values,
                                               stratify=train.label.values,
                                               random_state=42)

# Vectorizing

In [54]:
# vectorize tweets for model building
vectorizer=CountVectorizer(binary=True,stop_words='english')
# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train)+list(x_test))
# transform documents to document-term matrix
x_train_vec=vectorizer.transform(x_train)
x_test_vec=vectorizer.transform(x_test)

# Model building

In [55]:
# classify using support vector classifier
svm=svm.SVC(kernel='linear',probability=True)
# fit the SVC model based on the given training data
prob1=svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)
# perform classification and prediction on samples in x_test
y_pred_svm=svm.predict(x_test_vec)

# Accuracy score

In [56]:
print("Accuracy score for SVC is:",str(round(accuracy_score(y_test,y_pred_svm)*100, 2)),'%')

Accuracy score for SVC is: 95.07 %
